## Fitting drug response curves with sigmoid function

Save the dataset after filtering and fitting

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [4]:
def sigmoid_Wang(x, p, s):
    """ Sigmoid function from Dennis Wang's paper:
    x - dosage [0, 1],
    p - position,        default=0.4
    s - shape parameter, default=-1
    """
    return ( 1.0 / (1.0 + np.exp((x-p)/s)) )


def fsigmoid(x, p, k):
    """ Comparing with Dennis Wang's sigmoid:
    x = x  - dosage [0, 1]
    p - position [0,1],           default=0.4
    k = -1/s (s -shape parameter) default=0.4
    """
    return ( 1.0 / (1.0 + np.exp(-k*(x-p))) )


def sigmoid_4_param(x, x0, L, k, d):
    """ Comparing with Dennis Wang's sigmoid:
    x0 -  p - position, correlation with IC50 or EC50
    L = 1 in Dennis Wang's sigmoid, protect from devision by zero if x is too small 
    k = -1/s (s -shape parameter)
    d - determines the vertical position of the sigmoid - shift on y axis - better fitting then Dennis Wang's sigmoid
    """
    return ( 1/ (L + np.exp(-k*(x-x0))) + d)


def sigmoid_3_param(x, x0, k, d):
    """ Comparing with Dennis Wang's sigmoid:
    x0 -  p - position, correlation with IC50 or EC50
    k = -1/s (s -shape parameter)
    d - determines the vertical position of the sigmoid - shift on y axis - better fitting then Dennis Wang's sigmoid
        """
    return ( 1/ (1 + np.exp(-k*(x-x0))) + d )


def ll4(x, e, c, b, d):
    """ https://gist.github.com/yannabraham/5f210fed773785d8b638
    This function is basically a copy of the LL.4 function from the R drc package with
     - b: hill slope
     - d: min response - determines the vertical position of the graph
     - c: max response
     - e: EC50
     c-d - difference between max and min responses
     np.exp( b* (np.log(x)-np.log(e)) -  np.exp((x-p)/s in Dennis Wang's sigmoid
     b- hill slope = 1/s - shape parameter
     np.log(x)-np.log(e) == x-p in Dennis Wang's sigmoid
     """
    return ( (c-d)/(1 + np.exp( b*(np.log(x)-np.log(e) ))) + d)


def ll4_R(x, e, c, b, d):
    """ LL.4 function from R
    https://www.rdocumentation.org/packages/drc/versions/2.5-12/topics/LL.4
   
    c-d - difference between max and min responses
    np.exp( b* np.log(x) - e) -  np.exp((x-p)/s in Dennis Wang's sigmoid
    b - hill slope = 1/s - shape parameter
    np.log(x)- e/b == x-p in Dennis Wang's sigmoid
    """
    return ( (c-d)/(1+np.exp(b*np.log(x)- e)) + d)


def logistic4(x, A, B, C, d):
    """ https://people.duke.edu/~ccc14/pcfb/analysis.html
    4PL logistic equation
    Dennis Wang's sigmoid: 1.0 / (1.0 + np.exp((x-p)/s)
    (A - d) = 1 in Dennis Wang's sigmoid:
    (x/C)**B  - corresponds to np.exp((x-p)/s
    d - determines the vertical position of the graph
    """
    return ( (A-d)/(1.0+((x/C)**B)) + d )


def logLogistR(x, EC50, HS, E_inf):
    """Python analog for PharmacoGx/R/LogLogisticRegression.R
    https://github.com/bhklab/PharmacoGx/blob/master/R/LogLogisticRegression.R
    E = E_inf + (1 - E_inf)/(1 + (x/EC50)^HS)
    Dennis Wang's sigmoid: 1.0 / (1.0 + np.exp((x-p)/s)
    
    (A - d) = 1 in Dennis Wang's sigmoid:
    (np.log10(x)/EC50)**HS  - (in logistic4 (x/C)**B) corresponds to np.exp((x-p)/s 
    
    E_inf - determines the vertical position of the graph /coefficient d, min response in other functions
    """
    return ((1-E_inf)/(1+(np.log10(x)/EC50)**HS) + E_inf)



def FitCurve(fitting_function, x, y, parameters_guess=None, to_plot = False):
#     from scipy.optimize import curve_fit

    if parameters_guess:
        parameters, p_covariance = curve_fit(fitting_function, x, y, parameters_guess)
    else: 
        parameters, p_covariance = curve_fit(fitting_function, x, y)
    x2 = np.linspace(0, 1, 10)
    y_fit = fitting_function(x, *parameters)
    r2 = r2_score(y, y_fit)

    if to_plot:
        print("Fitting parameters:", *parameters)
        plt.scatter(x, y)
        x2 = np.linspace(0, 1, 10)
        y2 = fitting_function(x2, *parameters)
        plt.plot(x2, y2, "blue", label = "R^2= %0.5f"%r2)   
        plt.title('Least-squares fit')
        plt.legend();
    return r2, parameters


def FittingColumn(df, indexes, x_columns, y_columns, fitting_function, parameters_guess=None, default_param = False):
    """
    intial parameter guess [max(y), np.median(x), 1, min(y)]
    potentially they can be different for each data row, but as soon as we have scaled and filtered data
    we can use by default [1.0, 0.4, 1.0, .0] 
    """
    
    r2_scores = np.zeros(len(indexes))
    X = df.loc[indexes, x_columns].values.astype(np.float32)
    Y = df.loc[indexes, y_columns].values.astype(np.float32)
    fitting_parameters = [None]*len(indexes)
    
    
    # parameters_guess= [np.median(x), 1, max(y), min(y)]
#     default_param_model = {"sigmoid_Wang": [0.4, 0.1],
#                        "fsigmoid" : [0.4, -10],
#                        "sigmoid_4_param": [0.4, 1.0, 1.0, .0],
#                        "sigmoid_3_param": [0.4, 1.0, .0],
#                        "logistic4": [1.0, 1.0, 1.0, 0.0],
#                        "ll4": [0.4, 1.0, 1.0, 0.0],
#                        "ll4_R": [0.4, 1.0, 1.0, 0.0],
#                        "logLogistR": [-1, -0.1, 0.1]}
    
    default_param_model = {"sigmoid_Wang": [0.4, 0.1],
                       "fsigmoid" : [0.4, -10],
                       "sigmoid_4_param": [0.2, 0.8, -4.0, 0.2],
                       "sigmoid_3_param": [0.4, 1.0, .0],
                       "logistic4": [1.0, 1.0, 1.0, 0.0],
                       "ll4": [0.4, 1.0, 1.0, 0.0],
                       "ll4_R": [0.4, 1.0, 1.0, 0.0],
                       "logLogistR": [-1, -0.1, 0.1]}
    
    if default_param:
        parameters_guess = default_param_model[fitting_function]
       
    else:
        pass
    
    for i in tqdm(range(len(indexes))):
        x = X[i, :]
        y = Y[i, :]
    
        try:
            r2_scores[i], fitting_parameters[i] = FitCurve(fitting_function_object, x, y, parameters_guess = parameters_guess)
            
        except:
            try:
                functions = {"fsigmoid": fsigmoid, 
                 "sigmoid_Wang": sigmoid_Wang, 
                "sigmoid_4_param": sigmoid_4_param,
                 "sigmoid_3_param": sigmoid_3_param, 
                 "logistic4": logistic4,  
                 "ll4": ll4, 
                 "ll4_R":ll4_R,
                 "logLogistR":logLogistR}
                fitting_function_object = functions[fitting_function]
#                 from fitting_curves.py import fitting_function_object
                r2_scores[i], fitting_parameters[i] = FitCurve(fitting_function_object, x, y, parameters_guess = parameters_guess)
            except:
                r2_scores[i] = 0
    return r2_scores, fitting_parameters

def FittedData(df, x_columns, y_columns, fitting_function, parameters_guess=[], default_param = True):
    
    r2, fit_param = FittingColumn(df, df.index, x_columns, y_columns, fitting_function, default_param = True)
    df["fitting_r2"] = r2
    df["fitting_param"] = fit_param
    df= df[df["fitting_r2"]!=0]
    return df

## Read filtered data

In [6]:
drug_df = pd.read_csv("GDSC2_dose_response.csv")
drug_df.shape

(476291, 23)

In [7]:
drug_df.columns  #checking the columns

Index(['CELL_LINE_NAME', 'COSMIC_ID', 'DRUG_ID', 'DRUGID_COSMICID',
       'SCAN_DRUGID_COSMICID_KEY', 'MAX_CONC', 'FOLD_RANGE', 'fd_num_0',
       'fd_num_1', 'fd_num_2', 'fd_num_3', 'fd_num_4', 'fd_num_5', 'fd_num_6',
       'fd_num_7', 'norm_cells_0', 'norm_cells_1', 'norm_cells_2',
       'norm_cells_3', 'norm_cells_4', 'norm_cells_5', 'norm_cells_6',
       'norm_cells_7'],
      dtype='object')

In [8]:
#For GDSC2, only 7-concentrations
conc_columns= ["fd_num_"+str(i) for i in range(8)]
response_norm = ['norm_cells_'+str(i) for i in range(8)]
#For GDSC1, 9-concentrations
#conc_columns= ["fd_num_"+str(i) for i in range(10)]
#response_norm = ['norm_cells_'+str(i) for i in range(10)]

## Fit and save the data

In [9]:
fitting_function = "sigmoid_4_param"
df = FittedData(drug_df, x_columns=conc_columns, y_columns= response_norm, 
                fitting_function=fitting_function, default_param = True)
print("Fitted data with removed unfit", df.shape)

100%|██████████| 476291/476291 [46:59<00:00, 168.94it/s] 


Fitted data with removed unfit (342744, 25)


In [10]:
def SplitFitParam(df):
    """Column fitting_param is splitted into separate columns """
    conc_columns= ["fd_num_"+str(i) for i in range(8)]
    response_norm = ['norm_cells_'+str(i) for i in range(8)]
    param_columns = ["SCAN_DRUGID_COSMICID_KEY","DRUG_ID", "COSMIC_ID", "FOLD_RANGE", "MAX_CONC"] + conc_columns + response_norm
    for i in range(len(df['fitting_param'].values[0])):
        param_col = "param_"+str(i+1)
        param_columns.append(param_col)
        df[param_col] = df['fitting_param'].apply(lambda x: x[i])
    return df[param_columns]

In [11]:
df = SplitFitParam(df)
df

,SCAN_DRUGID_COSMICID_KEY,DRUG_ID,COSMIC_ID,FOLD_RANGE,MAX_CONC,fd_num_0,fd_num_1,fd_num_2,fd_num_3,fd_num_4,fd_num_5,fd_num_6,fd_num_7,norm_cells_0,norm_cells_1,norm_cells_2,norm_cells_3,norm_cells_4,norm_cells_5,norm_cells_6,norm_cells_7,param_1,param_2,param_3,param_4
0,14769_1003_924100_0,1003,924100,1000,0.1,0,0.142857,0.285714,0.428571,0.571429,0.714286,0.857143,1.0,1.0,0.958893,0.935710,1.000000,0.927973,0.763074,0.484846,0.386936,0.722313,1.655106,-14.439699,0.369405
1,24662_1003_924100_0,1003,924100,1000,0.1,0,0.142857,0.285714,0.428571,0.571429,0.714286,0.857143,1.0,1.0,0.972678,0.837348,0.956244,0.952168,0.826235,0.410892,0.345329,0.751288,1.665138,-24.564996,0.343845
2,24663_1003_924100_0,1003,924100,1000,0.1,0,0.142857,0.285714,0.428571,0.571429,0.714286,0.857143,1.0,1.0,0.982598,0.994612,1.000000,0.966509,0.677494,0.348320,0.468714,0.709173,1.723464,-135.008354,0.408517
3,32860_1003_924100_0,1003,924100,1000,0.1,0,0.142857,0.285714,0.428571,0.571429,0.714286,0.857143,1.0,1.0,1.000000,1.000000,0.962354,1.000000,0.656783,0.399142,0.333251,0.692795,1.564381,-25.023829,0.356669
4,32861_1003_924100_0,1003,924100,1000,0.1,0,0.142857,0.285714,0.428571,0.571429,0.714286,0.857143,1.0,1.0,0.970333,0.961104,1.000000,0.927306,0.619976,0.318820,0.410984,0.680177,1.623868,-25.228189,0.361407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476282,49717_2111_908450_0,2111,908450,1000,10.0,0,0.142857,0.285714,0.428571,0.571429,0.714286,0.857143,1.0,1.0,1.000000,0.920867,0.846594,0.857710,0.868865,0.891174,0.913444,0.262984,8.035821,-116.211565,0.875557
476283,49717_2169_908450_0,2169,908450,1000,10.0,0,0.142857,0.285714,0.428571,0.571429,0.714286,0.857143,1.0,1.0,0.883711,0.735166,0.928291,0.820593,0.772283,0.776014,0.883711,0.122035,5.535345,-110.577660,0.819343
476284,49717_2170_908450_0,2170,908450,1000,8.0,0,0.142857,0.285714,0.428571,0.571429,0.714286,0.857143,1.0,1.0,0.902289,1.000000,0.943176,0.917175,0.924598,0.865172,0.894866,0.297990,11.527230,-8.309338,0.880660
476286,49717_2172_908450_0,2172,908450,1000,10.0,0,0.142857,0.285714,0.428571,0.571429,0.714286,0.857143,1.0,1.0,0.998871,1.000000,0.809438,0.783437,0.620006,0.560580,0.449151,0.552697,1.476374,-5.020282,0.370441


In [12]:
df[df.columns[-10:]].head()

,norm_cells_2,norm_cells_3,norm_cells_4,norm_cells_5,norm_cells_6,norm_cells_7,param_1,param_2,param_3,param_4
0,0.935710,1.000000,0.927973,0.763074,0.484846,0.386936,0.722313,1.655106,-14.439699,0.369405
1,0.837348,0.956244,0.952168,0.826235,0.410892,0.345329,0.751288,1.665138,-24.564996,0.343845
2,0.994612,1.000000,0.966509,0.677494,0.348320,0.468714,0.709173,1.723464,-135.008354,0.408517
3,1.000000,0.962354,1.000000,0.656783,0.399142,0.333251,0.692795,1.564381,-25.023829,0.356669
4,0.961104,1.000000,0.927306,0.619976,0.318820,0.410984,0.680177,1.623868,-25.228189,0.361407


## Here we save the data with the parameter values that fit each curve

In [13]:
df.to_csv("GDSC2_fit_drug_profiles.csv", index=False)